## <strong>Sistema de 2ª ordem</strong>

A função de transferência de um **sistema de 2ª ordem** pode ser escrita da seguinte maneira:

$$G(s) = \frac{{\omega_n}^2}{s^2+2\zeta\omega_ns+{\omega_n}^2}$$

Sendo que: 

- $\omega_n$ é a frequência natural
- $\zeta$ é o coeficiente de amortecimento

Com esse sistema é obtida a seguinte **resposta ao degrau unitário**, no domínio do tempo:

$$c(t)=1-\frac{1}{\beta}e^{-\zeta\omega_nt}sen(\beta\omega_nt+\theta)$$

Sendo:  

$$\beta=\sqrt{1-\zeta^2}\hspace{1.5cm}\theta=tan^{-1}(\frac{\beta}{\zeta})$$

Além disso, através desses parâmetros da função de transferência, é possível determinar alguns \
 outros, para a **resposta ao degrau unitário**:

$$\tau = \frac{1}{\zeta\omega_n}\hspace{1.5cm}M_p = 1 + e^{\frac{-\zeta\pi}{\sqrt[]{1-\zeta^2}}}\hspace{1.5cm}T_s = \frac{4.6}{\zeta\omega_n}$$

Sendo que: 

- $\tau$ é a constante de tempo
- $M_p$ é o pico máximo ("overshoot")
- $T_s$ é uma estimativa do tempo de acomodação

O objetivo aqui é interagir com um sistema de segunda ordem, no formato apresentado acima,\
e observar a **resposta ao degrau**, o **plano s (com os polos do sistema)** e o **diagrama de Bode**.

In [ ]:
# Importa bibliotecas
import scipy as sp
from IPython.display import display
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.gridspec as gsp
import matplotlib.patches as pat
import numpy as np
from ipywidgets import interact
import warnings
%matplotlib inline

In [ ]:
#Sistema de referência
w_n_ref = 5
zeta_ref = 0.1 

num_ref = [w_n_ref**2]
den_ref = [1, 2*w_n_ref*zeta_ref, w_n_ref**2]

z_ref, p_ref, k_ref = signal.tf2zpk(num_ref, den_ref)

def segunda_ordem(w_n, zeta):

    font1 = {'color':'k','size': 14}
    font2 = {'color':'k','size': 12}           

    tau = 1/(zeta*w_n) 
    ts = 4.6/(zeta*w_n) 
    theta = np.rad2deg(np.arctan(np.sqrt(1-zeta**2)/zeta))
    t1 = np.linspace(0, 12, 1000)
    freq = np.linspace(0, 100, 1000)

    t_ref, resp_deg_ref = signal.step((z_ref, p_ref, k_ref), T=t1)
    w_ref, mag_ref, fas_ref = signal.bode((z_ref, p_ref, k_ref), freq)

    num = [w_n**2]
    den = [1, 2*w_n*zeta, w_n**2]

    z1, p1, k1 = signal.tf2zpk(num, den)

    plt.figure(figsize=(18,6))

    #Resposta ao degrau
    t, resp_deg = signal.step((z1, p1, k1), T=t1)

    plt.plot(t, resp_deg, c='r', label='Sistema G(s)')
    plt.plot(t_ref, resp_deg_ref, c='#C2C5CC', label='Sistema referência')

    if zeta<1:
      mp = 1 + np.exp(-zeta*np.pi/(np.sqrt(1-zeta**2)))
      tp = np.pi/(w_n*np.sqrt(1-zeta**2))
      plt.axhline(y=mp, xmin=0, xmax=tp/12, c='k', lw=1, ls='--')
      plt.axvline(x=tp, ymin=0, ymax=mp/2.2, c='k', lw=1, ls='--')
      plt.plot(tp, mp, marker='.', c='k', ms=10)
      plt.text(tp+0.1, mp+0.1, r'$M_p=1+e^{\frac{-\zeta\pi}{\sqrt{1-\zeta^2}}}=%.2f$'%mp, fontdict=font1)

    plt.axvline(x=ts, ymin=0, ymax=resp_deg[int(ts*1000/12)]/2.2, c='k', lw=1, ls='--')
    plt.text(ts-0.5, -0.25, r'$T_s = \frac{4.6}{\zeta\omega_n}=%.2f$'%ts, fontdict=font1)
    plt.axvline(x=tau, ymin=0, ymax=resp_deg[int(tau*1000/12)]/2.2, c='k', lw=1, ls='--')
    plt.text(tau-0.5, -0.25, r'$\tau = \frac{1}{\zeta\omega_n}=%.2f$'%tau, fontdict=font1)

    plt.title('Resposta ao degrau unitário')
    plt.xlabel("$t$ (s)")
    plt.ylabel("Amplitude")
    plt.ylim(0, 2.2)
    plt.xlim(0, 12)
    #plt.legend()

    plt.figure(figsize=(18,8.5))
    gs = gsp.GridSpec(2, 2)

    #Plano s
    plt.subplot(gs[:,1])
    plt.axhline(y=0, c='k', lw=0.5)
    plt.axvline(x=0, c='k', lw=0.5)
    plt.plot(p_ref.real, p_ref.imag, "x", mec='#C2C5CC', ms=15.0, label='Sistema referência') 
    plt.plot(p1.real, p1.imag, "x", c='r', ms=15.0, label='Sistema G(s)') 
    plt.xlim(-7,7)
    plt.ylim(-7,7)
    plt.title('Plano s')
    plt.xlabel("Real")
    plt.ylabel("Imaginário")

    plt.axvline(x=p1.real[0], ymin=(7+p1.imag[1])/14, ymax=(7+p1.imag[0])/14, c='k', lw=1, ls='--')
    plt.axhline(y=p1.imag[0], xmin=(7+p1.real[0])/14, xmax=0.5, c='k', lw=1, ls='--')
    plt.axhline(y=p1.imag[1], xmin=(7+p1.real[1])/14, xmax=0.5, c='k', lw=1, ls='--')
    plt.text(p1.real[0]-1, -0.5, r'%.2f'%p1.real[0], fontdict=font2)
    plt.text(0.1, p1.imag[0]-0.15, r'%.2f'%p1.imag[0], fontdict=font2)
    plt.text(0.1, p1.imag[1]-0.15, r'%.2f'%p1.imag[1], fontdict=font2)
    plt.text((p1.real[0]/2)-0.3, (p1.imag[0]/2)-0.3, r'$\omega_n = %.0f$'%w_n, fontdict=font2, rotation=-theta)
    plt.plot([0,p1.real[0]],[0,p1.imag[0]],c='k',lw=0.5)

    #Diagrama de Bode
    w, mag, fas = signal.bode((z1, p1, k1), freq)

    #Módulo
    plt.subplot(gs[0,0])
    plt.semilogx(w, mag, c='r')
    plt.semilogx(w_ref, mag_ref, c='#C2C5CC')
    plt.xlim(min(w), max(w))
    plt.title('Diagrama de Bode (módulo)')
    plt.xlabel("$f$ (rad/s)")
    plt.ylabel("$|G(j\omega)| (dB)$")
    plt.grid(True, which="both")

    #Fase
    plt.subplot(gs[1,0])
    plt.semilogx(w, fas, c='r')
    plt.semilogx(w_ref, fas_ref, c='#C2C5CC')
    plt.xlim(min(w), max(w))
    plt.title('Diagrama de Bode (fase)')
    plt.xlabel("$f$ (rad/s)")
    plt.ylabel("$\measuredangle G(j\omega)$ (deg)")
    plt.grid(True, which="both")

    plt.subplots_adjust(hspace = 0.5);

interact(segunda_ordem, w_n=(2, 6), zeta=(0.1, 1));
warnings.filterwarnings("ignore");

interactive(children=(IntSlider(value=4, description='w_n', max=6, min=2), FloatSlider(value=0.55, description…